In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import timedelta
import tqdm

In [2]:
records = [
    {
        'time': '2019-12-03 12:01:00',
        'reliability': 10,
        'R11':0.24,
        'R12':0.23,
        'R13':0,
        'R21':0.26,
        'R22':0.27,
        'R23':0,
        'R31':0,
        'R32':0,
        'R33':0,
    },
    {
        'time': '2019-12-03 12:02:00',
        'reliability': 8,
        'R11':0,
        'R12':0.33,
        'R13':0.16,
        'R21':0,
        'R22':0.4,
        'R23':0.11,
        'R31':0,
        'R32':0,
        'R33':0,
    },
    {
        'time': '2019-12-03 12:10:01',
        'reliability': 5,
        'R11':0,
        'R12':0,
        'R13':0,
        'R21':0,
        'R22':0.3,
        'R23':0.2,
        'R31':0,
        'R32':0.4,
        'R33':0.1,
    },
]

records = pd.DataFrame(records)

In [3]:
records.loc[:, 'time'] = pd.to_datetime(records.time)

records.head()

,time,reliability,R11,R12,R13,R21,R22,R23,R31,R32,R33
0,2019-12-03 12:01:00,10,0.24,0.23,0.00,0.26,0.27,0.00,0,0.0,0.0
1,2019-12-03 12:02:00,8,0.00,0.33,0.16,0.00,0.40,0.11,0,0.0,0.0
2,2019-12-03 12:10:01,5,0.00,0.00,0.00,0.00,0.30,0.20,0,0.4,0.1


In [4]:
np.unique(records.reliability, return_counts=True)

(array([ 5,  8, 10]), array([1, 1, 1]))

### Estimate posterior probabilities every 5 min (keep doing this up to T' min)

In [5]:
# change to historical data values
# P(I=1,R1)
def region_prior(region, time):
    # use labels_prv to prevent overfitting
    return {
        'R11':0.02,
        'R12':0.01,
        'R13':0.03,
        'R21':0.05,
        'R22':0.04,
        'R23':0.07,
        'R31':0.2,
        'R32':0.09,
        'R33':0.1,
    }[region]

np.sum([region_prior(r, 0) for r in np.unique(records.columns) if r.startswith('R')])

0.61

In [6]:
def p(r):
    return r / 10

In [7]:
def calculate_likelihood(m):
    c = m.reliability.apply(p)
    m.drop('reliability', axis=1, inplace=True)
    if m.shape[0] == 0:
        m.loc[0], c = np.zeros(m.shape[1]), 0
    return (m.T * c).T.prod(axis=0)

In [8]:
def calculate_posterior(df, r, time_step=5):
    time_steps_records = df.set_index('time').resample(f'{time_step}T')
    # Matrix - containes total likelihood
    matrix = time_steps_records.apply(calculate_likelihood)
    print('Likelihood:')
    print(matrix)
    print()
    # Find likelihood, prior for first time period
    likelihood = matrix.iloc[0, :]
    prior = np.array([region_prior(c, matrix.index[0]) for c in matrix])
    # Calcualate prior for I = 0 for first time period
    p_0 = (1 - np.sum(prior))
    # Calcualate likelihood for I = 0 for all periods using reliability of waze reports prod(1-p)
    l_0 = time_steps_records.apply(lambda m: (1 - m.reliability.apply(p)).prod())
    denom = np.sum(likelihood * prior) + l_0[0] * p_0
    # For each time step calculate posterior (start with zero row - special case) 
    print(f'\nl_0[0] * p_0: {l_0[0] * p_0}')
    print(f'\nlikelihood * prior: {likelihood * prior}')
    # matrix - containes posteriors [partly]   
    matrix.iloc[0, :] = likelihood * prior / denom
    p_0 = l_0[0] * p_0 / denom
    for ridx in range(1, matrix.shape[0]):
        prior = matrix.iloc[ridx - 1, :]
        likelihood = matrix.iloc[ridx, :]
        if np.sum(likelihood) == 0:
            matrix.iloc[ridx, :] = prior
            continue
        # Calculate denominator of posterior
        print(f'\nl_0[{ridx}] * p_0: {l_0[ridx] * p_0}')
        print(f'\nlikelihood * prior: {likelihood * prior}')
        denom = np.sum(likelihood * prior) + l_0[ridx] * p_0
        # Calculate posterior
        matrix.iloc[ridx, :] = likelihood * prior / denom
        p_0 = l_0[ridx] * p_0 / denom
    print()
    print(matrix)
    # Return posteriors
    return matrix.index, matrix.loc[:, r]

In [9]:
def incident_posterior(df, incident_interval=25):
    incident_interval = timedelta(minutes=incident_interval)
    features = []
    incident_id = 0
    for col in tqdm.tqdm(df.columns):
        # for each region
        if col.startswith('R22'):
            df_region = df.loc[df[col] > 0, :]
            incident_time = np.min(df_region.time)
            while incident_time <= np.max(df_region.time):
                incident_records = df_region[(incident_time <= df_region.time) & (df_region.time < (incident_time + incident_interval))]
                time_steps, posterior_probs = calculate_posterior(incident_records, col)
                features_temp = []
                for time_step, posterior_proba in zip(time_steps, posterior_probs):
                    features_temp.append({
                        'incident_id': incident_id, 
                        'start_time': incident_time,
                        'end_time': incident_time + incident_interval,
                        'time': time_step, 
                        'region': col,
                        'posterior_proba': posterior_proba,
                    })
                features += features_temp
                incident_time = np.min(df_region[(incident_time + incident_interval) <= df_region.time].time)
                incident_id = incident_id + 1
    return pd.DataFrame(features)

features = incident_posterior(records)

features.head()

100%|██████████| 11/11 [00:00<00:00, 92.01it/s]

Likelihood:
                     R11      R12  R13  R21     R22  R23  R31  R32   R33
time                                                                    
2019-12-03 12:00:00  0.0  0.06072  0.0  0.0  0.0864  0.0  0.0  0.0  0.00
2019-12-03 12:05:00  0.0  0.00000  0.0  0.0  0.0000  0.0  0.0  0.0  0.00
2019-12-03 12:10:00  0.0  0.00000  0.0  0.0  0.1500  0.1  0.0  0.2  0.05


l_0[0] * p_0: 0.0

likelihood * prior: R11    0.000000
R12    0.000607
R13    0.000000
R21    0.000000
R22    0.003456
R23    0.000000
R31    0.000000
R32    0.000000
R33    0.000000
Name: 2019-12-03 12:00:00, dtype: float64

l_0[2] * p_0: 0.0

likelihood * prior: R11    0.000000
R12    0.000000
R13    0.000000
R21    0.000000
R22    0.127584
R23    0.000000
R31    0.000000
R32    0.000000
R33    0.000000
dtype: float64

                     R11       R12  R13  R21       R22  R23  R31  R32  R33
time                                                                      
2019-12-03 12:00:00  0.0  0.149439  0.0  0.0  

,incident_id,start_time,end_time,time,region,posterior_proba
0,0,2019-12-03 12:01:00,2019-12-03 12:26:00,2019-12-03 12:00:00,R22,0.850561
1,0,2019-12-03 12:01:00,2019-12-03 12:26:00,2019-12-03 12:05:00,R22,0.850561
2,0,2019-12-03 12:01:00,2019-12-03 12:26:00,2019-12-03 12:10:00,R22,1.000000
